# Modeling the Future 2018

In [1]:
import math
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
sns.set()
import warnings
warnings.simplefilter('ignore',FutureWarning)

## Scenario Response Topic 2- Pharmacy Benefit

In [2]:
pharmacy = pd.read_excel('Scenario-Response-2-Pharmacy-Benefit-Data-Continuance-Table.xls')
pharmacy = pharmacy.rename({'Prescriptions (scripts)' : 'scripts', 
                            'Probability' : 'probability', 
                            'Cumulative Probability' : 'cumulative_probability'}, axis='columns')

<u>**Part 1: The Basics**</u>

**Question 1:** Using the Excel file, what are the expected number of scripts per person
per year?

In [37]:
avg_scripts = (pharmacy['probability'] * pharmacy['scripts']).sum()
avg_scripts

25.46743195135006

How many scripts will Company A likely insure during a year?

In [38]:
avg_scripts * 500000

12733715.975675032

**Question 2:** What is the expected number of scripts considering only members who
have at least one script?

In [40]:
have_scripts = pharmacy[pharmacy.scripts > 0]
(have_scripts['probability'] * have_scripts['scripts']).sum()

25.467431951350065

**Question 3:** What is the variance and standard deviation in the number of scripts per
person per year?

$$\sigma^2=\sum{(x-\bar{x})^2P(x)}$$

In [6]:
scripts_var = ((pharmacy['scripts'] - pharmacy['scripts'].mean()) ** 2 * pharmacy['probability']).sum()

In [7]:
scripts_var

962.6706220448247

In [8]:
scripts_std = math.sqrt(scripts_var)

In [9]:
scripts_std

31.02693381636066

<u>**Part 2: Severity**</u>

**Question 4:** If Company A covered 100% of the costs of the drugs, what is the pure
premium for a given member?

*Note: pure premium is the
likelihood the event will happen (called the frequency) multiplied by the insurer’s
expected cost when the event happens (called the severity)*

In [10]:
pp = (pharmacy['probability'] * ((pharmacy['scripts'] * 0.88 * 30) + (pharmacy['scripts'] * 0.12 * 120))).sum()

In [11]:
pp

1039.0712236150823

**Question 5:** Assume that the standard deviation of the cost of a script is $75.00. Using
the formula for variance below, calculate the minimum premium that allows a margin
equal to 1.5 standard deviations. 

Let S=N\*X where N=frequency, and X=severity. Let E[ ] represent the expected value of
what is in the brackets, Var( ) represent the variance of what is in the brackets, and S =
Pure Premium. Then one can show that if pure premium = E[S] = E[N]\*E[X], then Var(S) =
Var(X\*E[N] + Var(N)\*(E[X])^2

In [73]:
var_pp = (75 ** 2) * pharmacy['probability'].mean() + pharmacy['probability'].var() * (75 ** 2)

In [74]:
pp - var_pp * 1.5

951.9149343368813

<u>**Part 3: Insurance Benefits**</u>

**Question 6:** Suppose the only people who enroll in prescription-drug-only coverage
were those who expect to have at least 25 prescriptions a year. Revise the continuance
table and calculate the required pure premium.

In [75]:
pharmacy2 = pharmacy[pharmacy.scripts >= 25]
prob_total = pharmacy2['probability'].sum()
pharmacy2['probability'] = pharmacy2['probability'] / prob_total

In [76]:
pharmacy2['cumulative_probability'] = np.cumsum(pharmacy2['probability'])

In [77]:
pp2 = (pharmacy2['probability'] * ((pharmacy2['scripts'] * 0.88 * 30) + (pharmacy2['scripts'] * 0.12 * 120))).sum()
var_pp2 = (75 ** 2) * pharmacy2['probability'].mean() + pharmacy2['probability'].var() * (75 ** 2)

In [78]:
pp2 - var_pp2 * 1.5

1525.5189599139978

1. the member must pay \$5.00 for every generic script
2. the member must pay \$50.00 for every brand script

**Question 7:** Using the frequencies in the original continuance table, what is the new
pure premium given the above benefit structure?

In [79]:
pp_copay = (pharmacy['probability'] * ((pharmacy['scripts'] * 0.88 * 30 - 5) + (pharmacy['scripts'] * 0.12 * 120 - 50))).sum()

In [80]:
pp_copay

984.0712236150825

Company A is concerned that with the above copayments, a member could have very large
out-of-pocket costs. Therefore, Company A added a third benefit called an “out-of-pocket
maximum.” With the out of pocket maximum the company will cover all costs after a
member pays $500 in copayments.

**Question 8:** What is the new pure premium given the out-of-pocket maximum? Explain
the assumptions you made in order to do this calculation. Why might another actuary
calculate a different number that could also be reasonably defended?